<a href="https://colab.research.google.com/github/d-jenkins/NBA_Champs/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import all libraries needed
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
import json


In [ ]:
# list holding abbviations for all nba teams to create urls
teams = ['ATL', 'NJN', 'BOS', 'CHA', 'CHI', 
         'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
         'HOU', 'IND', 'LAC', 'LAL', 'MEM', 
         'MIA', 'MIL', 'MIN', 'NOH', 'NYK', 
         'OKC', 'ORL', 'PHI', 'PHO', 'POR', 
         'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

# array to hold all tables for all teams for all seasons
every_season = []

# iterate through list of all team abbreviations
for team in teams:

  # create url to scrape for team
  url = f'https://www.basketball-reference.com/teams/{team}'

  # scrape teams stats from their bball reference page
  stats = pd.read_html(url)[0]

  # select only the desired columns 
  stats = stats[["Team", "Season", "Rel Pace", "Rel ORtg", "Rel DRtg", "Playoffs"]]

  # manupulate select colums for team
  for i in range(0,len(stats["Playoffs"])):
    # remove asterisk from team
    stats["Team"][i] = stats["Team"][i].replace("*", "")

    # change playoff status to binary of if they won chip or not
    if stats["Playoffs"][i] == "Won Finals":
      stats["Playoffs"][i] = 1
    else:
      stats["Playoffs"][i] = 0

  # rename "playoffs" column to "Chip?"
  stats = stats.rename(columns={"Playoffs":"Chip?"})
  
  # Limit to all years since the 70s
  stats = stats.iloc[0:51, :]

  # add table of teams stats to a list of tables holding all teams stats
  every_season.append(stats)

# combine all teams stats into one dataframe
all_stats = pd.concat(every_season)


In [ ]:
# display all teams' stats
all_stats


,Team,Season,Rel Pace,Rel ORtg,Rel DRtg,Chip?
0,Atlanta Hawks,2020-21,-1.6,3.4,1.0,0
1,Atlanta Hawks,2019-20,2.7,-3.4,4.2,0
2,Atlanta Hawks,2018-19,3.9,-2.3,3.5,0
3,Atlanta Hawks,2017-18,1.0,-3.6,2.0,0
4,Atlanta Hawks,2016-17,1.0,-3.9,-3.1,0
...,...,...,...,...,...,...
46,Washington Bullets,1974-75,1.6,0.3,-6.4,0
47,Capital Bullets,1973-74,-1.5,-2.1,-3.5,0
48,Baltimore Bullets,1972-73,-2.0,-0.5,-3.5,0
49,Baltimore Bullets,1971-72,-1.5,-1.4,-0.3,0


In [ ]:
# select/adjust data to train ml model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

X = all_stats.drop(["Team", "Season", "Chip?"], axis=1)
y = all_stats["Chip?"].values.reshape(-1, 1)

# label-encode y data
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

# One-hot encodde y data
y = to_categorical(encoded_y)

y


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
# split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
# train random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf = rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))


0.9558823529411765


In [ ]:
# weighted importance of each stat towards winning a basketball game
sorted(zip(bf.feature_importances_, X.columns), reverse=True)

[(0.45078943560245877, 'Rel ORtg'),
 (0.4172580486702902, 'Rel DRtg'),
 (0.13195251572725109, 'Rel Pace')]

In [ ]:
# create array to hold binary model predictions
predictions = []

# convert hot encoded preditions back to binary values
for pred in rf.predict(X):
  if pred[1] == 0.:
    predictions.append(0)
  else:
    predictions.append(1)

# create new dataframe to hold stats with predictions
champs = all_stats
champs["Predicted"] = predictions

# show dataframe with all stats and championship predictions
champs

,Team,Season,Rel Pace,Rel ORtg,Rel DRtg,Chip?,Predicted
0,Atlanta Hawks,2020-21,-1.6,3.4,1.0,0,0
1,Atlanta Hawks,2019-20,2.7,-3.4,4.2,0,0
2,Atlanta Hawks,2018-19,3.9,-2.3,3.5,0,0
3,Atlanta Hawks,2017-18,1.0,-3.6,2.0,0,0
4,Atlanta Hawks,2016-17,1.0,-3.9,-3.1,0,0
...,...,...,...,...,...,...,...
46,Washington Bullets,1974-75,1.6,0.3,-6.4,0,0
47,Capital Bullets,1973-74,-1.5,-2.1,-3.5,0,0
48,Baltimore Bullets,1972-73,-2.0,-0.5,-3.5,0,0
49,Baltimore Bullets,1971-72,-1.5,-1.4,-0.3,0,0


In [ ]:
# create lists to hold overachieving and disappointing teams
overachievers = []
disappointments = []

# iterate through all seasons for all teams
for i in range(0, len(champs["Season"])):

  # if model said a team wouldnt win the chip but they do then add them to overachievers
  if (champs["Chip?"].values[i] == 1) and (champs["Predicted"].values[i] == 0):
    overachievers.append(f'{champs["Season"].values[i]} {champs["Team"].values[i]}')

  # if model said a team would win the chip but the don't then add them to disappointments
  elif (champs["Chip?"].values[i] == 0) and (champs["Predicted"].values[i] == 1):
    disappointments.append(f'{champs["Season"].values[i]} {champs["Team"].values[i]}')

# show both lists
print(overachievers)
print(disappointments)


['1975-76 New York Nets', '2007-08 Boston Celtics', '1975-76 Boston Celtics', '2015-16 Cleveland Cavaliers', '2010-11 Dallas Mavericks', '1988-89 Detroit Pistons', '2016-17 Golden State Warriors', '1987-88 Los Angeles Lakers', '1972-73 New York Knicks', '1978-79 Seattle SuperSonics', '2004-05 San Antonio Spurs', '1998-99 San Antonio Spurs', '2018-19 Toronto Raptors', '1977-78 Washington Bullets']
['1971-72 Milwaukee Bucks']


In [ ]:
type(0)

int

In [ ]:
# # tune hyperparameters
# from sklearn.model_selection import GridSearchCV

# # create new model whose hyperparamaters are to be tuned
# forest = RandomForestClassifier(random_state=42)

# # store some tuning options in a dictionary
# hyper = {'n_estimators': [10, 25, 50, 100, 200], 
#          'max_depth': [3, 5, 8, 15]}

# # create GridSearch model
# grid = GridSearchCV(forest, hyper, cv = 3, verbose = 1, n_jobs = -1)

# # Train the model with GridSearch
# fitted = grid.fit(X_train_scaled, y_train)

# # display best parameters and the score they get
# print(fitted.best_params_)
# print(fitted.best_score_)


In [ ]:
# best_forest = RandomForestClassifier(max_depth=3, n_estimators=10, random_state=42)
# bf = best_forest.fit(X_train_scaled, y_train)
# # **(why is score different than the "fitted" score in the cell above)
# print(bf.score(X_test_scaled, y_test))


In [ ]:
#import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt

#from matplotlib import style
#style.use("ggplot")
#from matplotlib import rcParams
#rcParams['figure.figsize'] = 10, 8

In [ ]:
#df = pd.dataFrame(os.path.join("..", "Chip?", "Season", "Team"))
#df.head()

In [ ]:
#target = df["Chip?"]
#target_names = ["Season", "team"]

In [ ]:
#data = df.drop("Chip?", axis=1)
#feature_names = data.columns
#data.head()

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
#from sklearn.svm import SVC 
#model = SVC(kernel='linear')
#model.fit(X_train, y_train)

In [ ]:
#print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
#from sklearn.metrics import classification_report
#predictions = model.predict(X_test)
#print(classification_report(y_test, predictions,
                            #target_names=target_names))

**ETL**

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.1'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graph

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NbaChamps").getOrCreate()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-06-09 03:29:16--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.36MB/s    in 0.2s    

2021-06-09 03:29:17 (4.36 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [ ]:
all_stats.dtypes

Team          object
Season        object
Rel Pace     float64
Rel ORtg     float64
Rel DRtg     float64
Chip?         object
Predicted      int64
dtype: object

In [ ]:
# Rename Columns for postgres consumption
all_stats_copy = all_stats.copy()
all_stats_py = all_stats_copy.rename(columns={"Rel Pace": "Rel_Pace", "Rel ORtg": "Rel_ORtg", "Rel DRtg": "Rel_DRtg", "Chip?": "Chip"})

# Convert Pandas df to Pyspark df
all_stats_py = spark.createDataFrame(all_stats_py)
all_stats_py.printSchema()
# all_stats_py

root
 |-- Team: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Rel_Pace: double (nullable = true)
 |-- Rel_ORtg: double (nullable = true)
 |-- Rel_DRtg: double (nullable = true)
 |-- Chip: long (nullable = true)
 |-- Predicted: long (nullable = true)



In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://nba-champs.c6ka6apltccn.us-east-2.rds.amazonaws.com:5432/nbachamps"
config = {"user":"postgres",
          "password": "MildredChase84!",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to Nba champs table in RDS

all_stats_py.write.jdbc(url=jdbc_url, table='all_stats', mode=mode, properties=config)

Py4JJavaError: ignored